In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.utils.data as Data
from IPython import display
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
num_inputs = 2
num_examples = 1000
true_w = [2,-3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

In [3]:
batch_size = 10
dataset = Data.TensorDataset(features, labels)
data_iter = Data.DataLoader(dataset = dataset, batch_size = batch_size, shuffle=True)

In [5]:
for X, y in data_iter:
    print(X); print(y)
    break

tensor([[-0.0657,  0.0511],
        [ 1.7841,  0.6601],
        [-0.8027, -1.3668],
        [-0.5571, -0.8237],
        [-0.5018, -1.1788],
        [-0.0675,  0.0740],
        [-0.7964,  0.8105],
        [ 0.7327,  2.7248],
        [-0.1367,  0.2497],
        [ 0.3762, -0.3640]])
tensor([ 3.8953,  5.5276,  7.2517,  5.8966,  7.2072,  3.8117, -0.1442, -3.5909,
         3.0715,  6.1934])


In [9]:
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    
    def forward(self, x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
if torch.cuda.is_available():
    print('cuda!')
    net = nn.DataParallel(net, device_ids=[0])
print(net)

cuda!
DataParallel(
  (module): LinearNet(
    (linear): Linear(in_features=2, out_features=1, bias=True)
  )
)


In [21]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
loss_func = nn.MSELoss().cuda()

num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        b_x = X.cuda(); b_y = y.cuda()
        output = net(b_x)
        loss = loss_func(output, b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('epoch %d, loss: %.4f'%(epoch, loss.data.cpu().numpy()))

D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:431: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 1, loss: 7.8013
epoch 2, loss: 14.2480
epoch 3, loss: 15.6459


In [22]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[ 0.1763, -0.3006]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([4.2822], device='cuda:0', requires_grad=True)
